In [4]:
from Data_manager.UserUtils import *
from Data_manager.split_functions.split_train_validation_random_holdout import \
    split_train_in_two_percentage_global_sample
URM_all = getURM_all()
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

In [5]:
from Hybrid import *
model = ScoresHybridRecommender

from Evaluation.Evaluator import EvaluatorHoldout
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10], ignore_users=[])

EvaluatorHoldout: Ignoring 2200 (17.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users


In [7]:
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Hybrid import LinearHybridRecommender, LinearZScoreNormalizedHybridRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender

model = LinearHybridRecommender
slim = SLIMElasticNetRecommender(URM_train)
rp3 = RP3betaRecommender(URM_train)
itemknn = ItemKNNCFRecommender(URM_train)
slim.fit(topK=8894, l1_ratio=0.05565733019999427, alpha=0.0012979360257937668)
rp3.fit(topK=32, alpha=0.22028996694465927, beta=0.17339063177346345)
#rp3.fit(topK=101, alpha=0.3026342852596128, beta=0.058468783118329024)
itemknn.fit(topK=31, shrink=435, similarity='tversky', normalize=True,
                                       feature_weighting='BM25', tversky_alpha=0.17113169506422393, tversky_beta=0.5684024974085575)
hybrid_1 = ScoresHybridRecommender(URM_train, slim, rp3, itemknn)
hybrid_1.fit(alpha=0.5072513954300486, beta=0.30902582824263064)
from Evaluation.Evaluator import EvaluatorHoldout
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10], ignore_users=[])

SLIMElasticNetRecommender: URM Detected 208 ( 1.6%) users with no interactions.
SLIMElasticNetRecommender: URM Detected 108 ( 0.5%) items with no interactions.
RP3betaRecommender: URM Detected 208 ( 1.6%) users with no interactions.
RP3betaRecommender: URM Detected 108 ( 0.5%) items with no interactions.
ItemKNNCFRecommender: URM Detected 208 ( 1.6%) users with no interactions.
ItemKNNCFRecommender: URM Detected 108 ( 0.5%) items with no interactions.
SLIMElasticNetRecommender: Processed 22222 (100.0%) in 4.56 min. Items per second: 81.17
RP3betaRecommender: Similarity column 22222 (100.0%), 7305.73 column/sec. Elapsed time 3.04 sec
Similarity column 22222 (100.0%), 9992.13 column/sec. Elapsed time 2.22 sec
ScoresHybridRecommender: URM Detected 208 ( 1.6%) users with no interactions.
ScoresHybridRecommender: URM Detected 108 ( 0.5%) items with no interactions.
EvaluatorHoldout: Ignoring 2200 (17.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 0 Users


In [8]:
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender

ease = EASE_R_Recommender(URM_train)
ease.fit(topK=24, l2_norm=37.54323189430143)
hybrid_2 = LinearHybridRecommender(URM_train, [slim, rp3, itemknn, ease])
hybrid_2.fit(alphas=[0.8857667100747117, 0.6379807942443021, 0.0690929184825888, 0.11953478623354052])

EASE_R_Recommender: URM Detected 208 ( 1.6%) users with no interactions.
EASE_R_Recommender: URM Detected 108 ( 0.5%) items with no interactions.
EASE_R_Recommender: Fitting model... 
EASE_R_Recommender: Fitting model... done in 2.15 min
SLIMElasticNetRP3betaItemKNNCFEASE_R_HybridRecommender: URM Detected 208 ( 1.6%) users with no interactions.
SLIMElasticNetRP3betaItemKNNCFEASE_R_HybridRecommender: URM Detected 108 ( 0.5%) items with no interactions.


In [9]:
import optuna as op
def objective(trial):
    alpha = trial.suggest_float("alpha", 0.0, 1.0)
    
    recommender = TwoScoresHybridRecommender(URM_train, hybrid_2, hybrid_1)
    recommender.fit(alpha=alpha)
    
    result, _ = evaluator_validation.evaluateRecommender(recommender)
    MAP_result = result["MAP"].item()
       
    return MAP_result

In [ ]:
best_params = {
    "alphas": [1, 1, 1],
}

study_name = "nested-hybrids-study"  # Unique identifier of the study.
storage_name = "sqlite:///db.db"
study = op.create_study(study_name=study_name, storage=storage_name, direction="maximize", load_if_exists=True)
#study.enqueue_trial(best_params)
study.optimize(objective, n_trials=100)

In [16]:
study.best_params

{'alpha': 0.5072513954300486, 'beta': 0.30902582824263064}

In [17]:
final = ScoresHybridRecommender(URM_train_validation, slim, rp3, itemknn)
final.fit(**study.best_params)

ScoresHybridRecommender: URM Detected 219 ( 1.7%) users with no interactions.
ScoresHybridRecommender: URM Detected 132 ( 0.6%) items with no interactions.


In [11]:
userknn = UserKNNCFRecommender(URM_train)
userknn.fit(topK=469, shrink=38, similarity='asymmetric', normalize=True,
                                       feature_weighting='TF-IDF', asymmetric_alpha=0.40077406933762383)

UserKNNCFRecommender: URM Detected 208 ( 1.6%) users with no interactions.
UserKNNCFRecommender: URM Detected 108 ( 0.5%) items with no interactions.
Similarity column 12638 (100.0%), 10007.60 column/sec. Elapsed time 1.26 sec


In [12]:
import optuna as op
def objective(trial):
    alpha = trial.suggest_float("alpha", 0.0, 1.0)
    
    recommender = TwoScoresHybridRecommender(URM_train, userknn, itemknn)
    recommender.fit(alpha=alpha)
    
    result, _ = evaluator_validation.evaluateRecommender(recommender)
    MAP_result = result["MAP"].item()
       
    return MAP_result

In [14]:
import numpy as np 
import scipy.sparse as sps
profile_length = np.ediff1d(sps.csr_matrix(URM_train).indptr)

#profile_length = np.ediff1d(sps.csr_matrix(URM_train_validation).indptr)
profile_length, profile_length.shape

(array([36, 30,  2, ...,  5, 17, 46], dtype=int32), (12638,))

In [24]:
# PROFILE LENGTH BASED GROUPING
from Evaluation.Evaluator import EvaluatorHoldout
import numpy as np

MAP_recommender_per_group = {}
cutoff = 10
threshold = 4

# Select users whose profile length is less than the cutoff for the group and greater than or equal to the cutoff for the previous group

users_in_group_flag = profile_length < threshold
users_in_group = np.arange(len(profile_length))[users_in_group_flag]

users_in_group_p_len = profile_length[users_in_group]

# Select users not in the current group
users_not_in_group_flag = np.isin(np.arange(len(profile_length)), users_in_group, invert=True)
users_not_in_group = np.arange(len(profile_length))[users_not_in_group_flag]

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[cutoff], ignore_users=users_not_in_group)


EvaluatorHoldout: Ignoring 2200 (17.4%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 10040 Users


In [25]:
study_name = "KNN-study"  # Unique identifier of the study.
storage_name = "sqlite:///db.db"
study = op.create_study(study_name=study_name, storage=storage_name, direction="maximize", load_if_exists=True)
study.optimize(objective, n_trials=100)

/var/folders/l3/n3yjvx_d2g9dy0ztkyft2th40000gn/T/ipykernel_78705/1549462082.py:3: FutureWarning: delete_study(): Please give all values as keyword arguments. See https://github.com/optuna/optuna/issues/3324 for details.
  op.delete_study(study_name, storage_name)
[I 2023-12-22 01:45:12,575] A new study created in RDB with name: KNN-study


TwoScoresHybridRecommender: URM Detected 208 ( 1.6%) users with no interactions.
TwoScoresHybridRecommender: URM Detected 108 ( 0.5%) items with no interactions.
EvaluatorHoldout: Processed 1141 (100.0%) in 0.58 sec. Users per second: 1958


[I 2023-12-22 01:45:13,191] Trial 0 finished with value: 0.007551889876605028 and parameters: {'alpha': 0.8095870605372276}. Best is trial 0 with value: 0.007551889876605028.


TwoScoresHybridRecommender: URM Detected 208 ( 1.6%) users with no interactions.
TwoScoresHybridRecommender: URM Detected 108 ( 0.5%) items with no interactions.
EvaluatorHoldout: Processed 1141 (100.0%) in 0.51 sec. Users per second: 2252


[I 2023-12-22 01:45:13,723] Trial 1 finished with value: 0.007551994212818041 and parameters: {'alpha': 0.05084823895978796}. Best is trial 1 with value: 0.007551994212818041.


...

[I 2023-12-22 01:46:05,668] Trial 97 finished with value: 0.007624438323386605 and parameters: {'alpha': 0.07817064220938542}. Best is trial 2 with value: 0.007721957903815912.


TwoScoresHybridRecommender: URM Detected 208 ( 1.6%) users with no interactions.
TwoScoresHybridRecommender: URM Detected 108 ( 0.5%) items with no interactions.
EvaluatorHoldout: Processed 1141 (100.0%) in 0.50 sec. Users per second: 2298


[I 2023-12-22 01:46:06,189] Trial 98 finished with value: 0.007660782104252731 and parameters: {'alpha': 0.09403625239892137}. Best is trial 2 with value: 0.007721957903815912.


TwoScoresHybridRecommender: URM Detected 208 ( 1.6%) users with no interactions.
TwoScoresHybridRecommender: URM Detected 108 ( 0.5%) items with no interactions.
EvaluatorHoldout: Processed 1141 (100.0%) in 0.50 sec. Users per second: 2277


[I 2023-12-22 01:46:06,714] Trial 99 finished with value: 0.007580999680035601 and parameters: {'alpha': 0.1689643441003334}. Best is trial 2 with value: 0.007721957903815912.
